In [70]:
def clip_score(val):
    if val > 4.0:
        return 4.0
    elif val < 1.0:
        return 1.0
    else:
        return val


def add_missing_entries(source_path, target_path):
    s_df = read_csv(source_path, sep=',', usecols=['pair_id'])
    t_df = read_csv(target_path, sep=',', usecols=['pair_id', 'Overall'])
    print(s_df['pair_id'].unique().shape)
    print(t_df['pair_id'].unique().shape)
    # print(t_df.shape)
    # n_df =
    # print(n_df.shape)
    final_rows = []
    for row in s_df.itertuples(index=False):
        p_id = row[0]
        t_row = t_df.loc[t_df['pair_id'] == p_id]
        score = t_row.iloc[0, 1] if t_row.shape[0] >= 1 else random.choice([2, 3])
        final_rows.append([p_id, score])

    df = DataFrame(final_rows, columns=['pair_id', 'Overall'])
    df['Overall'] = df['Overall'].apply(clip_score)
    df.to_csv(target_path, mode='w', index=False)

In [86]:
import numpy as np
import random

from sklearn.model_selection import train_test_split

from pandas import read_csv, concat, DataFrame
from pytorch_tabnet.tab_model import TabNetRegressor

from src.config import CLEANED_PATH, DataType, UNCLEANED_PATH, RAW_FILE, INFERENCE_FILE


train_path = CLEANED_PATH.format(data_type=DataType.train.name) + INFERENCE_FILE
test_path = CLEANED_PATH.format(data_type=DataType.test.name) + INFERENCE_FILE

train_df = read_csv(train_path, sep=',', usecols=['sentences_mean', 'sentences_min', 'sentences_max',
                                                  'sentences_med', 'title', 'n1_title_n2_text',
                                                  'n2_title_n1_text', 'n1_title_n1_text',
                                                  'n2_title_n2_text', 'start_para', 'end_para',
                                                  'ner', 'tf_idf', 'wmd_dist', 'overall'])


train_df = train_df.drop_duplicates()
test_df = read_csv(test_path, sep=',', usecols=['sentences_mean', 'sentences_min', 'sentences_max',
                                                'sentences_med', 'title', 'n1_title_n2_text',
                                                'n2_title_n1_text', 'n1_title_n1_text',
                                                'n2_title_n2_text', 'start_para', 'end_para',
                                                'ner', 'tf_idf', 'wmd_dist'])

y = train_df.pop('overall')
y = y.values
x = train_df.values
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.05, random_state=5)
print(train_x.shape, np.array(train_y).shape, val_x.shape, val_y.shape)

(4590, 14) (4590,) (242, 14) (242,)


In [26]:
from pytorch_tabnet.metrics import Metric
from scipy.stats import pearsonr


class Pearson(Metric):
    def __init__(self):
        self._name = "pearson"
        self._maximize = True

    def __call__(self, y_true, y_score):
        return pearsonr(y_pred, val_y)

In [87]:
def train():
    clf = TabNetRegressor(seed=5, n_d=16, n_independent=1, n_shared=2, n_steps=5, momentum=0.19, mask_type='entmax')  #TabNetRegressor()
    clf.fit(train_x, np.array(train_y).reshape(-1, 1), eval_set=[(val_x, np.array(val_y).reshape(-1, 1))])
    
    return clf

model = train()
# print(model.__dict__)

Device used : cuda
epoch 0  | loss: 11.84973| val_0_mse: 3.66417 |  0:00:00s
epoch 1  | loss: 1.79276 | val_0_mse: 1.47862 |  0:00:00s
epoch 2  | loss: 1.03512 | val_0_mse: 1.39096 |  0:00:00s
epoch 3  | loss: 0.73674 | val_0_mse: 1.05266 |  0:00:00s
epoch 4  | loss: 0.61881 | val_0_mse: 1.03978 |  0:00:00s
epoch 5  | loss: 0.56444 | val_0_mse: 0.91519 |  0:00:01s
epoch 6  | loss: 0.55235 | val_0_mse: 0.83656 |  0:00:01s
epoch 7  | loss: 0.53148 | val_0_mse: 0.89056 |  0:00:01s
epoch 8  | loss: 0.51278 | val_0_mse: 0.84334 |  0:00:01s
epoch 9  | loss: 0.50782 | val_0_mse: 0.82895 |  0:00:02s
epoch 10 | loss: 0.50769 | val_0_mse: 0.86019 |  0:00:02s
epoch 11 | loss: 0.50316 | val_0_mse: 0.8221  |  0:00:02s
epoch 12 | loss: 0.50278 | val_0_mse: 0.80893 |  0:00:02s
epoch 13 | loss: 0.50055 | val_0_mse: 0.89366 |  0:00:02s
epoch 14 | loss: 0.50715 | val_0_mse: 0.76424 |  0:00:02s
epoch 15 | loss: 0.49152 | val_0_mse: 0.78388 |  0:00:03s
epoch 16 | loss: 0.48084 | val_0_mse: 0.79238 |  0:00

In [80]:
pred_val = model.predict(val_x)


pred_val = np.squeeze(np.asarray(pred_val))

val_y = np.squeeze(np.asarray(val_y))
print(pearsonr(pred_val, val_y))

(0.7935352695459977, 4.0292371297862686e-158)


In [68]:
import wandb
sweep_config = {
  "name" : "tabnet-b-100",
    "metric": {"name": "pearson", "goal": "maximize"},
  "method" : "bayes",
  "parameters" : {
    "seed": {"values" :[5]},
    "n_d" : {"values" :[8, 12, 16, 20, 24]},
    "n_steps": {
        "min": 3,
        "max": 8
    },
    "n_independent": {
        "min": 1,
        "max": 5
    },
    "n_shared": {
        "min": 1,
        "max": 5
    },
    "momentum": {
        "min": 0.01,
        "max": 0.4
    },
    "mask_type": {"values" :["sparsemax", "entmax"]}
  }
}

sweep_id = wandb.sweep(sweep_config, project="SemEval-Task-8", entity="notsomonk")

Create sweep with ID: 4t2h3fjz
Sweep URL: https://wandb.ai/notsomonk/SemEval-Task-8/sweeps/4t2h3fjz


In [ ]:
def train():
    with wandb.init() as run:
        config = wandb.config
        model = make_model(config)
        for epoch in range(config["epochs"]):
            loss = model.fit()  # your model training code here
            wandb.log({"loss": loss, "epoch": epoch})

count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

In [69]:
def train():
    with wandb.init() as run:
        import numpy as np
        import random

        from sklearn.model_selection import train_test_split

        from pandas import read_csv, concat, DataFrame
        from pytorch_tabnet.tab_model import TabNetRegressor

        from src.config import CLEANED_PATH, DataType, UNCLEANED_PATH, RAW_FILE, INFERENCE_FILE


        train_path = CLEANED_PATH.format(data_type=DataType.train.name) + INFERENCE_FILE
        test_path = CLEANED_PATH.format(data_type=DataType.test.name) + INFERENCE_FILE

        train_df = read_csv(train_path, sep=',', usecols=['sentences_mean', 'sentences_min', 'sentences_max',
                                                          'sentences_med', 'title', 'n1_title_n2_text',
                                                          'n2_title_n1_text', 'n1_title_n1_text',
                                                          'n2_title_n2_text', 'start_para', 'end_para',
                                                          'ner', 'tf_idf', 'wmd_dist', 'overall'])


        train_df = train_df.drop_duplicates()
        test_df = read_csv(test_path, sep=',', usecols=['sentences_mean', 'sentences_min', 'sentences_max',
                                                        'sentences_med', 'title', 'n1_title_n2_text',
                                                        'n2_title_n1_text', 'n1_title_n1_text',
                                                        'n2_title_n2_text', 'start_para', 'end_para',
                                                        'ner', 'tf_idf', 'wmd_dist'])

        y = train_df.pop('overall')
        y = y.values
        x = train_df.values
        train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.15, random_state=5)
        
        config = wandb.config
        clf = TabNetRegressor(**config)  #TabNetRegressor()
        clf.fit(train_x, np.array(train_y).reshape(-1, 1), eval_set=[(val_x, np.array(val_y).reshape(-1, 1))])
        
        
        pred_val = clf.predict(val_x)
        
        pred_val = np.squeeze(np.asarray(pred_val))
        val_y = np.squeeze(np.asarray(val_y))
        
        p_s = pearsonr(pred_val, val_y)
        wandb.log({"pearson": p_s[0]})

count = 100 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: g9qnpr9p with config:
wandb: 	mask_type: sparsemax
wandb: 	momentum: 0.1795786550449812
wandb: 	n_d: 12
wandb: 	n_independent: 4
wandb: 	n_shared: 2
wandb: 	n_steps: 6
wandb: 	seed: 5
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Device used : cuda
epoch 0  | loss: 8.08177 | val_0_mse: 5.58359 |  0:00:00s
epoch 1  | loss: 2.29005 | val_0_mse: 1.73575 |  0:00:00s
epoch 2  | loss: 1.63158 | val_0_mse: 1.98248 |  0:00:00s
epoch 3  | loss: 1.195   | val_0_mse: 1.39432 |  0:00:01s
epoch 4  | loss: 1.11975 | val_0_mse: 1.21482 |  0:00:01s
epoch 5  | loss: 1.07678 | val_0_mse: 1.30251 |  0:00:01s
epoch 6  | loss: 0.81231 | val_0_mse: 1.46327 |  0:00:02s
epoch 7  | loss: 0.82965 | val_0_mse: 0.92023 |  0:00:02s
epoch 8  | loss: 0.77882 | val_0_mse: 0.93488 |  0:00:02s
epoch 9  | loss: 0.76152 | val_0_mse: 0.91054 |  0:00:03s
epoch 10 | loss: 0.75762 | val_0_mse: 0.9332  |  0:00:03s
epoch 11 | loss: 0.89413 | val_0_mse: 0.93136 |  0:00:04s
epoch 12 | loss: 1.06822 | val_0_mse: 0.99202 |  0:00:04s
epoch 13 | loss: 0.75124 | val_0_mse: 0.87063 |  0:00:04s
epoch 14 | loss: 0.79435 | val_0_mse: 0.98102 |  0:00:05s
epoch 15 | loss: 0.75398 | val_0_mse: 0.85802 |  0:00:05s
epoch 16 | loss: 0.71527 | val_0_mse: 0.95634 |  0:00

pearson,▁
pearson,0.77148


wandb: Agent Starting Run: bkjdokji with config:
wandb: 	mask_type: sparsemax
wandb: 	momentum: 0.08175560611527018
wandb: 	n_d: 12
wandb: 	n_independent: 1
wandb: 	n_shared: 5
wandb: 	n_steps: 7
wandb: 	seed: 5
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Device used : cuda
epoch 0  | loss: 10.36213| val_0_mse: 3.88142 |  0:00:00s
epoch 1  | loss: 2.50942 | val_0_mse: 1.85333 |  0:00:00s
epoch 2  | loss: 1.57743 | val_0_mse: 1.34098 |  0:00:01s
epoch 3  | loss: 1.13351 | val_0_mse: 1.04792 |  0:00:01s
epoch 4  | loss: 0.93672 | val_0_mse: 1.00733 |  0:00:01s
epoch 5  | loss: 0.8812  | val_0_mse: 1.03779 |  0:00:02s
epoch 6  | loss: 0.82557 | val_0_mse: 1.05669 |  0:00:02s
epoch 7  | loss: 0.74736 | val_0_mse: 0.9246  |  0:00:02s
epoch 8  | loss: 0.79937 | val_0_mse: 1.16297 |  0:00:03s
epoch 9  | loss: 0.76232 | val_0_mse: 0.92028 |  0:00:03s
epoch 10 | loss: 0.7474  | val_0_mse: 0.97491 |  0:00:03s
epoch 11 | loss: 0.65979 | val_0_mse: 0.96433 |  0:00:04s
epoch 12 | loss: 0.66773 | val_0_mse: 0.89734 |  0:00:04s
epoch 13 | loss: 0.65177 | val_0_mse: 1.00376 |  0:00:05s
epoch 14 | loss: 0.63155 | val_0_mse: 0.81489 |  0:00:05s
epoch 15 | loss: 0.6109  | val_0_mse: 0.76942 |  0:00:05s
epoch 16 | loss: 0.63709 | val_0_mse: 0.75191 |  0:00

...

Device used : cuda
epoch 0  | loss: 3.80639 | val_0_mse: 2.11262 |  0:00:00s
epoch 1  | loss: 1.64899 | val_0_mse: 1.76651 |  0:00:00s
epoch 2  | loss: 1.04094 | val_0_mse: 1.06621 |  0:00:00s
epoch 3  | loss: 0.86617 | val_0_mse: 1.12112 |  0:00:00s
epoch 4  | loss: 0.67812 | val_0_mse: 0.92533 |  0:00:00s
epoch 5  | loss: 0.616   | val_0_mse: 1.23796 |  0:00:00s
epoch 6  | loss: 0.59775 | val_0_mse: 0.94329 |  0:00:00s
epoch 7  | loss: 0.58956 | val_0_mse: 0.96335 |  0:00:00s
epoch 8  | loss: 0.57414 | val_0_mse: 0.95858 |  0:00:01s
epoch 9  | loss: 0.56769 | val_0_mse: 0.8854  |  0:00:01s
epoch 10 | loss: 0.58199 | val_0_mse: 0.99218 |  0:00:01s
epoch 11 | loss: 0.56838 | val_0_mse: 0.82031 |  0:00:01s
epoch 12 | loss: 0.55164 | val_0_mse: 1.00049 |  0:00:01s
epoch 13 | loss: 0.5717  | val_0_mse: 0.82253 |  0:00:01s
epoch 14 | loss: 0.56577 | val_0_mse: 0.93018 |  0:00:01s
epoch 15 | loss: 0.57045 | val_0_mse: 0.79328 |  0:00:01s
epoch 16 | loss: 0.55663 | val_0_mse: 0.78912 |  0:00

pearson,▁
pearson,0.76187


wandb: Agent Starting Run: lfhigajb with config:
wandb: 	mask_type: entmax
wandb: 	momentum: 0.08652274414839396
wandb: 	n_d: 20
wandb: 	n_independent: 2
wandb: 	n_shared: 2
wandb: 	n_steps: 5
wandb: 	seed: 5
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Device used : cuda
epoch 0  | loss: 8.55345 | val_0_mse: 2.19062 |  0:00:00s
epoch 1  | loss: 1.8744  | val_0_mse: 1.41998 |  0:00:00s
epoch 2  | loss: 1.25515 | val_0_mse: 1.28519 |  0:00:00s
epoch 3  | loss: 0.92188 | val_0_mse: 1.00506 |  0:00:00s
epoch 4  | loss: 0.78451 | val_0_mse: 0.90205 |  0:00:01s
epoch 5  | loss: 0.69143 | val_0_mse: 1.07184 |  0:00:01s
epoch 6  | loss: 0.83974 | val_0_mse: 0.85058 |  0:00:01s
epoch 7  | loss: 0.67997 | val_0_mse: 0.97235 |  0:00:01s
epoch 8  | loss: 0.6493  | val_0_mse: 0.8924  |  0:00:01s
epoch 9  | loss: 0.59879 | val_0_mse: 0.84196 |  0:00:01s
epoch 10 | loss: 0.61257 | val_0_mse: 0.98607 |  0:00:02s
epoch 11 | loss: 0.57475 | val_0_mse: 0.97152 |  0:00:02s
epoch 12 | loss: 0.57701 | val_0_mse: 0.85261 |  0:00:02s
epoch 13 | loss: 0.55984 | val_0_mse: 0.80188 |  0:00:02s
epoch 14 | loss: 0.53838 | val_0_mse: 0.85077 |  0:00:02s
epoch 15 | loss: 0.54793 | val_0_mse: 0.79177 |  0:00:03s
epoch 16 | loss: 0.56157 | val_0_mse: 0.98684 |  0:00

wandb: Ctrl + C detected. Stopping sweep.


In [88]:
testp = model.predict(test_df.values)
testp = np.clip(testp, 1.0, 4.0)

In [90]:
test_df_full = read_csv(test_path, sep=',', usecols=['pair_id'])
test_df_full['Overall'] = testp
out_path = 'results/try/tabnet_95.csv'


test_df_full.to_csv(out_path, mode='w', columns=["pair_id", "Overall"], index=False)
UNCLEANED_PATH = UNCLEANED_PATH.format(data_type=DataType.test.name)
RAW_FILE = RAW_FILE.format(data_type=DataType.test.name)
add_missing_entries(UNCLEANED_PATH + RAW_FILE, out_path)

(4902,)
(4878,)
